# 1. Load Dataset

## Audio data
    - wav data와 이와 매칭되는 json label data를 model input이 가능하게 변환 필요
    - WhisperFeatureExtractor는 16kHz로 sampling된 데이터를 입력받는다


In [1]:
# set GPU
# torch 보다 먼저 import 해야 아래 사항이 적용된다
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "1"  # Set the GPU 1 to use
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


In [2]:
import glob
from datasets import load_dataset

# File path
training_raw_path = "/data/freetalk_senior/1.Training/raw_data/1.AI챗봇/1.AI챗봇_2_자유대화(노인남여)_TRAINING/"
training_labeled_path = "/data/freetalk_senior/1.Training/labeled_data/1.AI챗봇/1.AI챗봇_라벨링_자유대화(노인남여)_TRAINING/"
training_dir = [
    "노인남여_노인대화12_M_1534748575_63_강원_실내",
    "노인남여_노인대화13_F_1527537495_69_강원_실내",
    "노인남여_노인대화12_M_1554478480_62_수도권_실내",
    "노인남여_노인대화13_F_1533490517_65_수도권_실내",
    "노인남여_노인대화13_M_1532565861_61_전라_실내",
    "노인남여_노인대화15_F_1535863537_62_전라_실내",
    "노인남여_노인대화13_F_1555169713_66_기타_실내",
    "노인남여_노인대화15_F_1540391606_71_기타_실내",
    "노인남여_노인대화14_M_1542738257_68_경상_실내",
    "노인남여_노인대화15_F_1534589663_68_경상_실내",
    "노인남여_노인대화14_M_1531684542_70_충청_실내",
    "노인남여_노인대화14_F_1534566769_66_충청_실내"
]
#----------------------------------------------------------------------------------------------------------------
validation_raw_path = "/data/freetalk_senior/2.Validation/raw_data/1.AI챗봇/1.AI챗봇_자유대화(노인남여)_VALIDATION"
validation_labeled_path = "/data/freetalk_senior/2.Validation/labeled_data/1.AI챗봇/1.AI챗봇_라벨링_자유대화(노인남여)_VALIDATION"
validation_dir = [
    "노인남여_노인대화07_M_1527639004_76_강원_실내",#500
    "노인남여_노인대화07_F_1526732595_65_강원_실내",
    "노인남여_노인대화07_M_1520916170_75_수도권_실내",
    "노인남여_노인대화07_F_1520511716_63_수도권_실내",
    "노인남여_노인대화12_M_1532565861_61_전라_실내",
    "노인남여_자유대화_F_1521502420_62_전라_실내",
    "노인남여_노인대화08_F_1524884567_60_기타_실내",
    "노인남여_노인대화07_M_1527671642_80_경상_실내",
    "노인남여_노인대화08_F_1529841959_62_경상_실내",
    "노인남여_노인대화08_M_1528952078_70_충청_실내",
    "노인남여_노인대화08_F_1527825984_71_충청_실내",
]
#----------------------------------------------------------------------------------------------------------------
training_raw_data = []
training_raw_dir = [path + "/*.wav" for path in training_dir]
for d in training_raw_dir:
    training_raw_data.append(sorted(glob.glob(os.path.join(training_raw_path, d), recursive=True)))

training_labeled_data = []
training_labeled_dir = [path + "/*.json" for path in training_dir]
for d in training_labeled_dir:
    training_labeled_data.append(sorted(glob.glob(os.path.join(training_labeled_path, d), recursive=True)))
#----------------------------------------------------------------------------------------------------------------
validation_raw_data = []
validation_raw_dir = [path + "/*.wav" for path in validation_dir]
for d in validation_raw_dir:
    validation_raw_data.append(sorted(glob.glob(os.path.join(validation_raw_path, d), recursive=True)))

validation_labeled_data = []
validation_labeled_dir = [path + "/*.json" for path in validation_dir]
for d in validation_labeled_dir:
    validation_labeled_data.append(sorted(glob.glob(os.path.join(validation_labeled_path, d), recursive=True)))

In [3]:
print(f"TRAINING:   raw:{sum(len(data) for data in training_raw_data)}, label:{sum(len(data) for data in training_labeled_data)}")
print(f"VALIDATION: raw:{sum(len(data) for data in validation_raw_data)}, label:{sum(len(data) for data in validation_labeled_data)}")

TRAINING:   raw:13394, label:13394
VALIDATION: raw:9944, label:9944


In [49]:
data_files = {
    "train_input": training_raw_data,    
    "train_label": training_labeled_data,
    "validation_input": validation_raw_data,    
    "validation_label": validation_labeled_data,
    
}
len(training_raw_data[10]+ training_raw_data[9])

1782

In [39]:
# Load Dataset
from datasets import Features, Audio
import datasets 

# load_dataset 이 새로 만들어 버리는 label column 때문에 에러 발생하여 features명시
features = Features({
    'audio': Audio()
})

train_input = []
for d in data_files['train_input']:
    train_input.append(load_dataset("audiofolder", 
                           data_files=d, 
                           features=features)
                      )
train_input = [x['train'] for x in train_input]
train_input = datasets.DatasetDict({
    "train": datasets.concatenate_datasets(train_input, axis=0)
})

train_label = []
for d in data_files['train_label']:
    train_label.append(load_dataset("json", data_files=d)
                      )
train_label = [x['train'] for x in train_label]

train_label = datasets.DatasetDict({
    "train": datasets.concatenate_datasets(train_label, axis=0)
})

['/data/freetalk_senior/1.Training/raw_data/1.AI챗봇/1.AI챗봇_2_자유대화(노인남여)_TRAINING/노인남여_노인대화12_M_1534748575_63_강원_실내/노인남여_노인대화12_M_1534748575_63_강원_실내_15850.wav', '/data/freetalk_senior/1.Training/raw_data/1.AI챗봇/1.AI챗봇_2_자유대화(노인남여)_TRAINING/노인남여_노인대화12_M_1534748575_63_강원_실내/노인남여_노인대화12_M_1534748575_63_강원_실내_15851.wav', '/data/freetalk_senior/1.Training/raw_data/1.AI챗봇/1.AI챗봇_2_자유대화(노인남여)_TRAINING/노인남여_노인대화12_M_1534748575_63_강원_실내/노인남여_노인대화12_M_1534748575_63_강원_실내_15852.wav', '/data/freetalk_senior/1.Training/raw_data/1.AI챗봇/1.AI챗봇_2_자유대화(노인남여)_TRAINING/노인남여_노인대화12_M_1534748575_63_강원_실내/노인남여_노인대화12_M_1534748575_63_강원_실내_15853.wav', '/data/freetalk_senior/1.Training/raw_data/1.AI챗봇/1.AI챗봇_2_자유대화(노인남여)_TRAINING/노인남여_노인대화12_M_1534748575_63_강원_실내/노인남여_노인대화12_M_1534748575_63_강원_실내_15854.wav', '/data/freetalk_senior/1.Training/raw_data/1.AI챗봇/1.AI챗봇_2_자유대화(노인남여)_TRAINING/노인남여_노인대화12_M_1534748575_63_강원_실내/노인남여_노인대화12_M_1534748575_63_강원_실내_15855.wav', '/data/freetalk_senior/1.Training/raw_data/1.

Resolving data files:   0%|          | 0/500 [00:00<?, ?it/s]

['/data/freetalk_senior/1.Training/raw_data/1.AI챗봇/1.AI챗봇_2_자유대화(노인남여)_TRAINING/노인남여_노인대화13_F_1527537495_69_강원_실내/노인남여_노인대화13_F_1527537495_69_강원_실내_16254.wav', '/data/freetalk_senior/1.Training/raw_data/1.AI챗봇/1.AI챗봇_2_자유대화(노인남여)_TRAINING/노인남여_노인대화13_F_1527537495_69_강원_실내/노인남여_노인대화13_F_1527537495_69_강원_실내_16261.wav', '/data/freetalk_senior/1.Training/raw_data/1.AI챗봇/1.AI챗봇_2_자유대화(노인남여)_TRAINING/노인남여_노인대화13_F_1527537495_69_강원_실내/노인남여_노인대화13_F_1527537495_69_강원_실내_16270.wav', '/data/freetalk_senior/1.Training/raw_data/1.AI챗봇/1.AI챗봇_2_자유대화(노인남여)_TRAINING/노인남여_노인대화13_F_1527537495_69_강원_실내/노인남여_노인대화13_F_1527537495_69_강원_실내_16274.wav', '/data/freetalk_senior/1.Training/raw_data/1.AI챗봇/1.AI챗봇_2_자유대화(노인남여)_TRAINING/노인남여_노인대화13_F_1527537495_69_강원_실내/노인남여_노인대화13_F_1527537495_69_강원_실내_16278.wav', '/data/freetalk_senior/1.Training/raw_data/1.AI챗봇/1.AI챗봇_2_자유대화(노인남여)_TRAINING/노인남여_노인대화13_F_1527537495_69_강원_실내/노인남여_노인대화13_F_1527537495_69_강원_실내_16282.wav', '/data/freetalk_senior/1.Training/raw_data/1.

Resolving data files:   0%|          | 0/556 [00:00<?, ?it/s]

['/data/freetalk_senior/1.Training/raw_data/1.AI챗봇/1.AI챗봇_2_자유대화(노인남여)_TRAINING/노인남여_노인대화12_M_1554478480_62_수도권_실내/노인남여_노인대화12_M_1554478480_62_수도권_실내_14750.wav', '/data/freetalk_senior/1.Training/raw_data/1.AI챗봇/1.AI챗봇_2_자유대화(노인남여)_TRAINING/노인남여_노인대화12_M_1554478480_62_수도권_실내/노인남여_노인대화12_M_1554478480_62_수도권_실내_14751.wav', '/data/freetalk_senior/1.Training/raw_data/1.AI챗봇/1.AI챗봇_2_자유대화(노인남여)_TRAINING/노인남여_노인대화12_M_1554478480_62_수도권_실내/노인남여_노인대화12_M_1554478480_62_수도권_실내_14752.wav', '/data/freetalk_senior/1.Training/raw_data/1.AI챗봇/1.AI챗봇_2_자유대화(노인남여)_TRAINING/노인남여_노인대화12_M_1554478480_62_수도권_실내/노인남여_노인대화12_M_1554478480_62_수도권_실내_14753.wav', '/data/freetalk_senior/1.Training/raw_data/1.AI챗봇/1.AI챗봇_2_자유대화(노인남여)_TRAINING/노인남여_노인대화12_M_1554478480_62_수도권_실내/노인남여_노인대화12_M_1554478480_62_수도권_실내_14754.wav', '/data/freetalk_senior/1.Training/raw_data/1.AI챗봇/1.AI챗봇_2_자유대화(노인남여)_TRAINING/노인남여_노인대화12_M_1554478480_62_수도권_실내/노인남여_노인대화12_M_1554478480_62_수도권_실내_14755.wav', '/data/freetalk_senior/1.Training

...

Resolving data files:   0%|          | 0/896 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/500 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/556 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1600 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
senior_dataset = datasets.DatasetDict({
    "train": datasets.concatenate_datasets([train_input['train'], 
                                            train_label['train']], axis=1)
})

print(senior_dataset)

In [7]:
# Load Dataset
from datasets import Features, Audio
import datasets 

validation_input = []
for d in data_files['validation_input']:
    validation_input.append(load_dataset("audiofolder", 
                           data_files=d, 
                           features=features)
                      )
validation_input = [x['train'] for x in validation_input]

Resolving data files:   0%|          | 0/1324 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1599 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/480 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1486 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/676 [00:00<?, ?it/s]

...

Resolving data files:   0%|          | 0/1501 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/84 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/200 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/198 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1134 [00:00<?, ?it/s]

In [8]:
validation_input = datasets.DatasetDict({
    "validation": datasets.concatenate_datasets(validation_input, axis=0)
})

validation_label = []
for d in data_files['validation_label']:
    validation_label.append(load_dataset("json", data_files=d))
    
validation_label = [x['train'] for x in validation_label]

validation_label = datasets.DatasetDict({
    "validation": datasets.concatenate_datasets(validation_label, axis=0)
})

senior_dataset["validation"] = datasets.concatenate_datasets([validation_input['validation'], 
                                                              validation_label['validation']], axis=1)


Resolving data files:   0%|          | 0/1324 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1599 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/480 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1486 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/676 [00:00<?, ?it/s]

...

Resolving data files:   0%|          | 0/1501 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/84 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/200 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/198 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1134 [00:00<?, ?it/s]

In [9]:
senior_dataset = senior_dataset.shuffle(seed=44)

###### senior_dataset

# 2. Set Toknizer & FeatureExtractor

## Feature Extractor: Audio raw data에 대 preprocessing 한다
    - Transformers.WhisperFeatureExtractor
    1. Audio sample에 대해 padding 및 truncating을 해 모든 sample길이를 30초 맞춘다.
        whisper model은 attention mask없이 발화 신호로부터 무시해야 할 input을 직접 추론하도록 학습되었다
    2. Audio array를 log-Mel spectrum으로 변환. -> 이 것이 model input

## Model: seq-to-seq mapping

## Tokeninzer: model output을 text format으로 postprocessing
    - Transformers.WhisperTokenizer


In [10]:
checkpoint = "jiwon65/whisper-small_korean-zeroth"

# Whisper model output은 vocabulary 단어의 index
# 이를 실제 문자와 mapping하기 위해 Tokenizer 사용
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained(checkpoint, language="Korean", task="transcribe")

# Feature Extractor
from transformers import WhisperFeatureExtractor

# jiwon65/whisper-small_korean-zeroth model을 fine tuning
feature_extractor = WhisperFeatureExtractor.from_pretrained(checkpoint)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
# WhisperProcessor: 위 두 모듈을 하나로 묶는다
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(checkpoint, language="Korean", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# 3. Dataset Preprocessing

In [12]:
def preprocess_dataset(batch):
    audio=batch["audio"]
    label=batch["발화정보"]
    
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    batch["labels"] = tokenizer(label['stt']).input_ids
    return batch

senior_dataset = senior_dataset.map(preprocess_dataset, 
                                    remove_columns=senior_dataset.column_names["train"], 
                                    # batched=True,
                                    # batch_size=1,
                                    # num_proc=os.cpu_count()
                                   )

In [13]:
splited_senior_dataset = datasets.DatasetDict()

In [14]:
splited_senior_dataset = senior_dataset["train"].train_test_split(test_size=0.2)

In [15]:
# splited_senior_dataset["validation"] = senior_dataset["validation"]

In [16]:
splited_senior_dataset

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 10715
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 2679
    })
})

In [17]:
senior_dataset["validation"]

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 9944
})

# 4. Set Trainer & Training

## 4-1. Set DataCollator

    seq-to-seq model을 위한 DataCollator는 input_feature와 label을 독립적으로 다룬다는 점에서 특징적.
        input_feature -> FeatureExtractor
        label -> Tokenizer 로 다뤄야 함

In [18]:
# input_feature -> log_mel_spectrogram -> PyTorch Tensor
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union

# DatasetDict -> torch.tensor
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # 인풋 데이터와 라벨 데이터의 길이가 다르며, 따라서 서로 다른 패딩 방법이 적용되어야 한다. 그러므로 두 데이터를 분리해야 한다.
        # 먼저 오디오 인풋 데이터를 간단히 토치 텐서로 반환하는 작업을 수행한다.
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        
        # Tokenize된 레이블 시퀀스를 가져온다.
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # 레이블 시퀀스에 대해 최대 길이만큼 패딩 작업을 실시한다.
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        
        # 패딩 토큰을 -100으로 치환하여 loss 계산 과정에서 무시되도록 한다.
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        
        # 이전 토크나이즈 과정에서 bos 토큰이 추가되었다면 bos 토큰을 잘라낸다.
        # 해당 토큰은 이후 언제든 추가할 수 있다.
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]
        
        batch["labels"] = labels
        
        return batch

In [19]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(checkpoint, language="Korean", task="transcribe")

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## 4-2. Set metric

In [20]:
import evaluate

metric = evaluate.load('cer')

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    label_ids[label_ids == -100] = tokenizer.pad_token_id

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    cer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

def preprocess_logits_for_metrics(logits, labels):
    print(logits[0].shape)
    pred_ids = logits[0].argmax(dim=-1)
    return pred_ids




## 4-3. Set Train args

    output_dir: model weight를 저장하는 경로
    generation_max_length: evaluation동안 recursive하게 생성되는 token들의 최대 길이 
    save_steps: Training동안, 이 parameter에 설정한 step 마다 checkpoint 저장
    eval_steps: Training동안, 이 parameter에 설정한 step 마다 checkpoint 평가 수행
    report_to: training_log를 어디 저장할 지 설정. default: tensorboard

In [21]:
# pretrained_model load
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(checkpoint)

In [22]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [23]:
from transformers import Seq2SeqTrainingArguments
steps = 1000
training_args = Seq2SeqTrainingArguments(
    output_dir="./test_trainer",       # output dir
    max_steps=steps,                   # epoch 대신 설정
    evaluation_strategy="steps",       # evaluate after each "steps"

    warmup_steps=int(steps/40),             # num of warmup steps for learning rate scheduler
    save_steps=int(steps/20),
    eval_steps=int(steps/20),
    logging_steps=int(steps/50),            # how often to print log
    
    per_device_train_batch_size=64,    # batch size of per_device during 
    per_device_eval_batch_size=32,      # batch size per GPU for evaluation
    learning_rate=1e-5,                # initial lr for AdamW optimizer
    gradient_checkpointing=True,       # saving mem, but make backward pass slower
    fp16=True,                         # use mixed precision

    report_to=["tensorboard"],
    load_best_model_at_end=True,       # best checkpoint will always be saved
    
    metric_for_best_model="cer",       # 한국어의 경우 띄어쓰기가 애매한 경우가 많아서 'wer'보다는 'cer'이 더 적합할 것
    greater_is_better=False,           # is better metric result is big?
    push_to_hub=False,                   # Perform evaluation
    run_name="Training",       # experiment name
)




/home/wooseok/anaconda3/envs/whisper_kor/lib/python3.9/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [24]:
from transformers import Seq2SeqTrainingArguments
steps = 1000
training_args = Seq2SeqTrainingArguments(
    output_dir="./test_trainer",       # output dir
    # logging_dir="./logging_dir",       # storing to log_dir
    max_steps=steps,                   # epoch 대신 설정
    # num_train_epochs = 5,              # total num of training epochs
    evaluation_strategy="steps",       # evaluate after each "steps"

    warmup_steps=int(steps/40),             # num of warmup steps for learning rate scheduler
    save_steps=int(steps/20),
    eval_steps=int(steps/20),
    logging_steps=int(steps/50),            # how often to print log
    
    per_device_train_batch_size=8,    # batch size of per_device during 
    per_device_eval_batch_size=8,      # batch size per GPU for evaluation
    # per_device_eval_batch_size=10,   # batch size for evaluation
    gradient_accumulation_steps=2,     # tot num of steps before back propagation. train time increase.
    learning_rate=1e-5,                # initial lr for AdamW optimizer
    # weight_decay=0.01,                 # strength of weight decay
    gradient_checkpointing=True,       # saving mem, but make backward pass slower
    fp16=True,                         # use mixed precision
    # fp16_opt_level="02",               # mixed precision mode
    
    # predict_with_generate=True,        # using generate to calculate 'generative metrics'(ROUGE, BLEU)
    # generation_max_length=225,         # generative metrics's max len

    report_to=["tensorboard"],
    load_best_model_at_end=True,       # best checkpoint will always be saved
    
    metric_for_best_model="cer",       # 한국어의 경우 띄어쓰기가 애매한 경우가 많아서 'wer'보다는 'cer'이 더 적합할 것
    greater_is_better=False,           # is better metric result is big?
    push_to_hub=False,
    # do_train=True,                   # Perform training
    # do_eval=True,                    # Perform evaluation
    run_name="test1800Training",       # experiment name
    # seed=3,                            # seed for experiment reproductbility 3x3
)

/home/wooseok/anaconda3/envs/whisper_kor/lib/python3.9/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## 4-4. Set Trainer

In [25]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=splited_senior_dataset["train"],
    # eval_dataset=senior_dataset["validation"],
    eval_dataset=splited_senior_dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
)


max_steps is given, it will override any value given in num_train_epochs


## 4-5. Set GPU

In [26]:
# 맨 처음 set

In [27]:
print("CUDA_DEVICE_ORDER:", os.environ.get("CUDA_DEVICE_ORDER"))
print("CUDA_VISIBLE_DEVICES:", os.environ.get("CUDA_VISIBLE_DEVICES"))

CUDA_DEVICE_ORDER: PCI_BUS_ID
CUDA_VISIBLE_DEVICES: 1


In [28]:
# torch 를 먼저 import 하면 CUDA_VISIBLE_DEVICES 설정이 안먹음
import torch
print("Number of GPUs available:", torch.cuda.device_count())

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

model.config.use_cache = False # mem usage 줄이기 위함

Number of GPUs available: 1


In [29]:
print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Device: cuda:0
Current cuda device: 0
Count of using GPUs: 1


In [30]:
torch.cuda.empty_cache()

# trainer.train() # data 1800 개

# Evaluation

In [31]:
model_tuned = WhisperForConditionalGeneration.from_pretrained("./train_result_last/checkpoint-350/")

In [32]:
trainer_for_test_tuned = Seq2SeqTrainer(
    args=training_args,
    model=model_tuned,
    train_dataset=splited_senior_dataset["train"],
    eval_dataset=senior_dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
)

max_steps is given, it will override any value given in num_train_epochs


In [33]:
trainer_for_test_tuned.evaluate()

torch.Size([8, 26, 51865])


torch.Size([8, 26, 51865])
torch.Size([8, 26, 51865])
torch.Size([8, 26, 51865])
torch.Size([8, 35, 51865])
torch.Size([8, 34, 51865])
torch.Size([8, 26, 51865])
torch.Size([8, 29, 51865])
torch.Size([8, 30, 51865])
torch.Size([8, 29, 51865])
torch.Size([8, 25, 51865])
torch.Size([8, 28, 51865])
torch.Size([8, 33, 51865])
torch.Size([8, 29, 51865])
torch.Size([8, 27, 51865])
torch.Size([8, 28, 51865])
torch.Size([8, 29, 51865])
torch.Size([8, 24, 51865])
torch.Size([8, 33, 51865])
torch.Size([8, 24, 51865])
torch.Size([8, 29, 51865])
torch.Size([8, 24, 51865])
torch.Size([8, 27, 51865])
torch.Size([8, 28, 51865])
torch.Size([8, 35, 51865])
torch.Size([8, 24, 51865])
torch.Size([8, 26, 51865])
torch.Size([8, 32, 51865])
torch.Size([8, 25, 51865])
torch.Size([8, 23, 51865])
torch.Size([8, 24, 51865])
torch.Size([8, 28, 51865])
torch.Size([8, 29, 51865])
torch.Size([8, 28, 51865])
torch.Size([8, 28, 51865])
torch.Size([8, 29, 51865])
torch.Size([8, 32, 51865])
torch.Size([8, 27, 51865])
t

{'eval_loss': 0.39952999353408813,
 'eval_cer': 11.218262212126847,
 'eval_runtime': 1541.967,
 'eval_samples_per_second': 6.449,
 'eval_steps_per_second': 0.806}

In [34]:
model_baseline = WhisperForConditionalGeneration.from_pretrained(checkpoint)
model_baseline = model_baseline.to(device)

In [35]:
trainer_for_test_baseline = Seq2SeqTrainer(
    args=training_args,
    model=model_baseline,
    train_dataset=splited_senior_dataset["train"],
    eval_dataset=senior_dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
)

max_steps is given, it will override any value given in num_train_epochs


In [36]:
trainer_for_test_baseline.evaluate()

torch.Size([8, 26, 51865])


torch.Size([8, 26, 51865])
torch.Size([8, 26, 51865])
torch.Size([8, 26, 51865])
torch.Size([8, 35, 51865])
torch.Size([8, 34, 51865])
torch.Size([8, 26, 51865])
torch.Size([8, 29, 51865])
torch.Size([8, 30, 51865])
torch.Size([8, 29, 51865])
torch.Size([8, 25, 51865])
torch.Size([8, 28, 51865])
torch.Size([8, 33, 51865])
torch.Size([8, 29, 51865])
torch.Size([8, 27, 51865])
torch.Size([8, 28, 51865])
torch.Size([8, 29, 51865])
torch.Size([8, 24, 51865])
torch.Size([8, 33, 51865])
torch.Size([8, 24, 51865])
torch.Size([8, 29, 51865])
torch.Size([8, 24, 51865])
torch.Size([8, 27, 51865])
torch.Size([8, 28, 51865])
torch.Size([8, 35, 51865])
torch.Size([8, 24, 51865])
torch.Size([8, 26, 51865])
torch.Size([8, 32, 51865])
torch.Size([8, 25, 51865])
torch.Size([8, 23, 51865])
torch.Size([8, 24, 51865])
torch.Size([8, 28, 51865])
torch.Size([8, 29, 51865])
torch.Size([8, 28, 51865])
torch.Size([8, 28, 51865])
torch.Size([8, 29, 51865])
torch.Size([8, 32, 51865])
torch.Size([8, 27, 51865])
t

{'eval_loss': 0.8550595045089722,
 'eval_cer': 27.924403236627892,
 'eval_runtime': 1518.2538,
 'eval_samples_per_second': 6.55,
 'eval_steps_per_second': 0.819}

In [37]:
trainer.evaluate()

torch.Size([8, 44, 51865])


torch.Size([8, 46, 51865])
torch.Size([8, 26, 51865])
torch.Size([8, 46, 51865])
torch.Size([8, 24, 51865])
torch.Size([8, 43, 51865])
torch.Size([8, 40, 51865])
torch.Size([8, 28, 51865])
torch.Size([8, 51, 51865])
torch.Size([8, 24, 51865])
torch.Size([8, 39, 51865])
torch.Size([8, 50, 51865])
torch.Size([8, 47, 51865])
torch.Size([8, 38, 51865])
torch.Size([8, 42, 51865])
torch.Size([8, 42, 51865])
torch.Size([8, 38, 51865])
torch.Size([8, 42, 51865])
torch.Size([8, 23, 51865])
torch.Size([8, 38, 51865])
torch.Size([8, 21, 51865])
torch.Size([8, 61, 51865])
torch.Size([8, 35, 51865])
torch.Size([8, 29, 51865])
torch.Size([8, 31, 51865])
torch.Size([8, 28, 51865])
torch.Size([8, 37, 51865])
torch.Size([8, 28, 51865])
torch.Size([8, 51, 51865])
torch.Size([8, 24, 51865])
torch.Size([8, 34, 51865])
torch.Size([8, 23, 51865])
torch.Size([8, 34, 51865])
torch.Size([8, 28, 51865])
torch.Size([8, 38, 51865])
torch.Size([8, 32, 51865])
torch.Size([8, 44, 51865])
torch.Size([8, 25, 51865])
t

{'eval_loss': 0.6718575358390808,
 'eval_cer': 26.28211586901763,
 'eval_runtime': 434.6426,
 'eval_samples_per_second': 6.164,
 'eval_steps_per_second': 0.771}

In [38]:
splited_senior_dataset

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 10715
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 2679
    })
})